# DS-practice[2] Data Parsing and Cleansing

[학습목표] <b><span style="color:blue"> <u>Raw Data</u>에서 필요한 부분을 정제하여 추출할 수 있다. </span></b>

- <b>Text</b> Data Cleansing
    - Dropping Observation: <I> pandas.dropna </I>
    - Dropping Variable: <I> pandas.drop </I>
    - Filling the missing values



- <b>CSV</b> Data Parsing
    - above average data
    - above median data
    - the data from quantile(25%) to quantile(75%)

1) <b> Data Cleansing </b>

- Sample Data Generation: by using "StringIO"

In [2]:
import pandas as pd
import numpy as np

from io import StringIO

csv_data = StringIO("""
x1,x2,x3,x4,x5
1,0.1,"1",2019-01-01,A
2,,,2019-01-02,B
3,,"3",2019-01-03,C
,0.4,"4",2019-01-04,A
5,0.5,"5",2019-01-05,B
,,,2019-01-06,C
7,0.7,"7",,A
8,0.8,"8",2019-01-08,B
9,0.9,,2019-01-09,C
""")

data=pd.read_csv(csv_data,dtype={"x1":pd.Int64Dtype(),"x3":pd.Int64Dtype()},parse_dates=[3]) 
#x4가 날짜 형태이므로 parse_date라는 data를 날짜로 받을건지에 대해 [3]을 입력해준다


In [3]:
data #data에 csv파일 형태로 들어가게 된다

,x1,x2,x3,x4,x5
0,1,0.1,1,2019-01-01,A
1,2,NaN,<NA>,2019-01-02,B
2,3,NaN,3,2019-01-03,C
3,<NA>,0.4,4,2019-01-04,A
4,5,0.5,5,2019-01-05,B
5,<NA>,NaN,<NA>,2019-01-06,C
6,7,0.7,7,NaT,A
7,8,0.8,8,2019-01-08,B
8,9,0.9,<NA>,2019-01-09,C


- Data Observation

In [5]:
data.isnull() #각 요소마다 null인지 확인가능
data.isnull().sum() #각 column별로 null의 개수를 합하여서 나타내준다.

x1    2
x2    3
x3    3
x4    1
x5    0
dtype: int64

- Dropping Observation

In [6]:
data.dropna() #결측값(null)을 갖고있던 '행'들이 사라졌다. axis의 default가 0이였기 때문에 null존재 행을 삭제.

,x1,x2,x3,x4,x5
0,1,0.1,1,2019-01-01,A
4,5,0.5,5,2019-01-05,B
7,8,0.8,8,2019-01-08,B


- Dropping Variables

In [7]:
data.dropna(axis=1)  #axis를 1로 설정시 null을 갖고있던 '열'이 사라졌다

,x5
0,A
1,B
2,C
3,A
4,B
5,C
6,A
7,B
8,C


- Filling missing values

In [35]:
data = pd.DataFrame({'x': ['A', 'A', 'C', '', 'A', 'B', 'A', 'B', 'A', 'A', 'C',
                           '', 'A', 'A', '', 'A', 'B', 'A', 'C', 'A', '']}) #중간에 결측값 존재

In [9]:
data

,x
0,A
1,A
2,C
3,
4,A
5,B
6,A
7,B
8,A
9,A


In [12]:
np.random.uniform(0,1,10)  #0부터 1 사이의 10개의 난수 생성.

array([0.66867171, 0.18525708, 0.16988966, 0.64339837, 0.07807646,
       0.29235254, 0.24082621, 0.27002193, 0.05838168, 0.60585722])

In [36]:
def cat_fill_na():  #0~1사이 난수를 하나 생성하고 이 값이 크기에 따라 A,B,C를 설정하여 결측값을 채운다.
    rnd_num=np.random.uniform(0,1,1)
    if rnd_num>0.8:
        x='C'
    elif rnd_num>0.5:
        x='B'
    else:
        x='A'
    return x

In [37]:
for i in range(data.shape[0]): #data의 크기만큼 for문이 돈다.
    if data['x'].iloc[i]=='':  #data를 보면 결측값이 ''이므로 공백과 비교연산이 가능
        data['x'].iloc[i]=cat_fill_na() 
               
               

In [38]:
data

,x
0,A
1,A
2,C
3,A
4,A
5,B
6,A
7,B
8,A
9,A


2) <b> Data Parsing </b>

- parsing in SampleACC.csv data

In [3]:
data = pd.read_csv("SampleACC.csv") 

In [4]:
data

,nSeq,DI,PORT4_CHN1,PORT6_CHN1,PORT7_CHN1
0,0,0,-0.2087,-0.4806,0.3770
1,1,0,-0.2104,-0.4819,0.3759
2,2,0,-0.2103,-0.4822,0.3798
3,3,0,-0.2119,-0.4819,0.3806
4,4,0,-0.2148,-0.4812,0.3823
...,...,...,...,...,...
30445,30445,0,-0.0059,-0.6162,0.1777
30446,30446,0,-0.0034,-0.6157,0.1780
30447,30447,0,-0.0037,-0.6145,0.1772
30448,30448,0,0.0000,-0.6157,0.1820


- Parsing “the above-average data"

In [5]:
#bool대수 형태로 각 요소가 평균보다 큰지 작은지 T/F로 알려준다  (열 이름앞에 존재하는 띄워쓰기도 주의)
data[' PORT4_CHN1'] > np.mean(data[' PORT4_CHN1'])

0        False
1        False
2        False
3        False
4        False
         ...  
30445    False
30446    False
30447    False
30448     True
30449     True
Name:  PORT4_CHN1, Length: 30450, dtype: bool

In [6]:
#다음과 같이 data[]안에 넣어줘야 true값에 대해 출력을 해준다
data[data[' PORT4_CHN1'] > np.mean(data[' PORT4_CHN1'])]

,nSeq,DI,PORT4_CHN1,PORT6_CHN1,PORT7_CHN1
897,897,0,0.0029,0.2679,0.3245
898,898,0,0.0222,0.2749,0.2930
899,899,0,0.0415,0.2803,0.2649
900,900,0,0.0620,0.2848,0.2327
901,901,0,0.0791,0.2905,0.2021
...,...,...,...,...,...
29214,29214,0,0.0127,-0.6182,0.2725
29215,29215,0,0.0083,-0.6184,0.2673
29216,29216,0,0.0029,-0.6182,0.2581
30448,30448,0,0.0000,-0.6157,0.1820


- Parsing “the above-median data”

In [7]:
#평균대신 중간값을 이용하여 검색
data[data[' PORT4_CHN1'] > np.median(data[' PORT4_CHN1'])]

,nSeq,DI,PORT4_CHN1,PORT6_CHN1,PORT7_CHN1
893,893,0,-0.0737,0.2275,0.4470
894,894,0,-0.0544,0.2394,0.4167
895,895,0,-0.0371,0.2533,0.3848
896,896,0,-0.0171,0.2615,0.3538
897,897,0,0.0029,0.2679,0.3245
...,...,...,...,...,...
30445,30445,0,-0.0059,-0.6162,0.1777
30446,30446,0,-0.0034,-0.6157,0.1780
30447,30447,0,-0.0037,-0.6145,0.1772
30448,30448,0,0.0000,-0.6157,0.1820


- Parsing “the data from quantile(25%) to quantile(75%)"

In [8]:
A= np.quantile(data[' PORT4_CHN1'],0.25)
B= np.quantile(data[' PORT4_CHN1'],0.75)
#numpy에서 np.quantile함수를 이용하여 ' PORT4_CHN1'의 0.25와 0.75되는 지점의 값을 얻음

In [9]:
print(A,B)   #data에서 25%에 해당하는 값과 75%에 해당하는 값을 반환

-0.2732 0.2778


In [10]:
#numpy에서 지원하는 np.logical_and연산을 이용하여 and연산을 진행  -> ' PORT4_CHN1'값들 중 25%이상 75%이하에 해당하는 값만 저장
data[np.logical_and(data[' PORT4_CHN1']>A, data[' PORT4_CHN1']<B)]

,nSeq,DI,PORT4_CHN1,PORT6_CHN1,PORT7_CHN1
0,0,0,-0.2087,-0.4806,0.3770
1,1,0,-0.2104,-0.4819,0.3759
2,2,0,-0.2103,-0.4822,0.3798
3,3,0,-0.2119,-0.4819,0.3806
4,4,0,-0.2148,-0.4812,0.3823
...,...,...,...,...,...
30445,30445,0,-0.0059,-0.6162,0.1777
30446,30446,0,-0.0034,-0.6157,0.1780
30447,30447,0,-0.0037,-0.6145,0.1772
30448,30448,0,0.0000,-0.6157,0.1820
